In [1]:
import pandas as pd
from libcomcat.dataframes import (get_detail_data_frame, get_dyfi_data_frame,
                                  get_history_data_frame, get_magnitude_data_frame,
                                  get_pager_data_frame, get_phase_dataframe,
                                  get_summary_data_frame)
from libcomcat.search import search, get_event_by_id
from datetime import datetime

In [2]:
import os

filepaths = [f for f in os.listdir(".") if f.endswith('.csv')]
df = pd.concat(map(pd.read_csv, filepaths))

In [3]:
event_list = df['event_id'].tolist()
df_list = []
event_id_list = []

In [4]:
df['url'][0]

'https://earthquake.usgs.gov/earthquakes/eventpage/usp000ennh'

In [5]:
df = df.drop(['Unnamed: 0'], axis=1)

In [6]:
df.dtypes

event_id         object
datetime         object
location         object
latitude        float64
longitude       float64
depth           float64
magnitude       float64
significance      int64
alert            object
url              object
eventtype        object
country          object
dtype: object

In [7]:
event_dtypes = {
        'event_id' : str,
        'location' : str,
        'latitude' : float,
        'longitude' : float,
        'depth' : float,
        'magnitude' : float,
        'significance' : pd.Int64Dtype(),
        'alert' : str, 
        'url' : str,
        'eventtype' : str,
        'country' : str,
    }

df = df.astype(event_dtypes)

In [8]:
pd.to_datetime(df['datetime'])

0        2006-07-15 07:36:21.500
1        2008-04-03 19:07:55.890
2        2010-01-13 08:14:49.100
3        2011-08-27 16:15:29.700
4        2014-03-07 17:31:41.200
                   ...          
737439   2019-11-07 01:58:34.323
737440   2021-02-01 16:55:28.961
737441   2021-08-09 01:39:40.685
737442   2022-05-24 09:14:59.331
737443   2022-08-02 20:02:04.428
Name: datetime, Length: 737444, dtype: datetime64[ns]

In [9]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [10]:
df['date'] = df['datetime'].dt.date
df['timestamp'] = df['datetime'].dt.time
df = df.drop(['url','eventtype'], axis=1)

In [11]:
df

,event_id,datetime,location,latitude,longitude,depth,magnitude,significance,alert,country,date,timestamp
0,usp000ennh,2006-07-15 07:36:21.500,"76 km NNE of Punto Fijo, Venezuela",12.3300,-69.9200,25.800,4.4,298,nan,ABW,2006-07-15,07:36:21.500000
1,usp000g320,2008-04-03 19:07:55.890,"64 km NNE of Punto Fijo, Venezuela",12.2310,-69.9710,10.800,4.0,246,nan,ABW,2008-04-03,19:07:55.890000
2,usp000h62c,2010-01-13 08:14:49.100,"60 km NNE of Punto Fijo, Venezuela",12.2090,-70.0180,12.200,4.2,271,nan,ABW,2010-01-13,08:14:49.100000
3,usp000j74j,2011-08-27 16:15:29.700,"101 km NNE of Punto Fijo, Venezuela",12.4800,-69.7200,15.000,4.7,407,nan,ABW,2011-08-27,16:15:29.700000
4,usc000n5bb,2014-03-07 17:31:41.200,"67 km NNE of Punto Fijo, Venezuela",12.2700,-70.0000,23.600,3.0,139,nan,ABW,2014-03-07,17:31:41.200000
...,...,...,...,...,...,...,...,...,...,...,...,...
737439,us60006bx5,2019-11-07 01:58:34.323,"61 km ENE of Sinazongwe, Zambia",-16.9928,27.9628,5.130,4.5,312,nan,ZWE,2019-11-07,01:58:34.323000
737440,us6000ddq7,2021-02-01 16:55:28.961,"35 km E of Chipinge, Zimbabwe",-20.1780,32.9603,10.000,4.3,284,nan,ZWE,2021-02-01,16:55:28.961000
737441,us6000f3h6,2021-08-09 01:39:40.685,"9 km ESE of Binga, Zimbabwe",-17.6416,27.4238,10.000,4.3,284,nan,ZWE,2021-08-09,01:39:40.685000
737442,us7000hc68,2022-05-24 09:14:59.331,"14 km SE of Binga, Zimbabwe",-17.6952,27.4567,10.000,4.7,343,nan,ZWE,2022-05-24,09:14:59.331000


In [16]:
import numpy as np

In [19]:
conditions = [
    (df['magnitude'] >= 1) & (df['magnitude'] <= 2.9),
    (df['magnitude'] >= 3) & (df['magnitude'] <= 3.9),
    (df['magnitude'] >= 4) & (df['magnitude'] <= 4.9),
    (df['magnitude'] >= 5) & (df['magnitude'] <= 5.9),
    (df['magnitude'] >= 6) & (df['magnitude'] <= 6.9),
    (df['magnitude'] >= 7) & (df['magnitude'] <= 7.9),
    (df['magnitude'] >= 8)]

# create a list of the values we want to assign for each condition
values = ['micro', 'minor', 'light', 'moderate', 'strong', 'major', 'great']

# create a new column and use np.select to assign values to it using our lists as arguments
df['level'] = np.select(conditions, values)

# display updated DataFrame
df.head()

,event_id,datetime,location,latitude,longitude,depth,magnitude,significance,alert,country,date,timestamp,level
0,usp000ennh,2006-07-15 07:36:21.500,"76 km NNE of Punto Fijo, Venezuela",12.330,-69.920,25.8,4.4,298,nan,ABW,2006-07-15,07:36:21.500000,light
1,usp000g320,2008-04-03 19:07:55.890,"64 km NNE of Punto Fijo, Venezuela",12.231,-69.971,10.8,4.0,246,nan,ABW,2008-04-03,19:07:55.890000,light
2,usp000h62c,2010-01-13 08:14:49.100,"60 km NNE of Punto Fijo, Venezuela",12.209,-70.018,12.2,4.2,271,nan,ABW,2010-01-13,08:14:49.100000,light
3,usp000j74j,2011-08-27 16:15:29.700,"101 km NNE of Punto Fijo, Venezuela",12.480,-69.720,15.0,4.7,407,nan,ABW,2011-08-27,16:15:29.700000,light
4,usc000n5bb,2014-03-07 17:31:41.200,"67 km NNE of Punto Fijo, Venezuela",12.270,-70.000,23.6,3.0,139,nan,ABW,2014-03-07,17:31:41.200000,minor


In [20]:
df

,event_id,datetime,location,latitude,longitude,depth,magnitude,significance,alert,country,date,timestamp,level
0,usp000ennh,2006-07-15 07:36:21.500,"76 km NNE of Punto Fijo, Venezuela",12.3300,-69.9200,25.800,4.4,298,nan,ABW,2006-07-15,07:36:21.500000,light
1,usp000g320,2008-04-03 19:07:55.890,"64 km NNE of Punto Fijo, Venezuela",12.2310,-69.9710,10.800,4.0,246,nan,ABW,2008-04-03,19:07:55.890000,light
2,usp000h62c,2010-01-13 08:14:49.100,"60 km NNE of Punto Fijo, Venezuela",12.2090,-70.0180,12.200,4.2,271,nan,ABW,2010-01-13,08:14:49.100000,light
3,usp000j74j,2011-08-27 16:15:29.700,"101 km NNE of Punto Fijo, Venezuela",12.4800,-69.7200,15.000,4.7,407,nan,ABW,2011-08-27,16:15:29.700000,light
4,usc000n5bb,2014-03-07 17:31:41.200,"67 km NNE of Punto Fijo, Venezuela",12.2700,-70.0000,23.600,3.0,139,nan,ABW,2014-03-07,17:31:41.200000,minor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
737439,us60006bx5,2019-11-07 01:58:34.323,"61 km ENE of Sinazongwe, Zambia",-16.9928,27.9628,5.130,4.5,312,nan,ZWE,2019-11-07,01:58:34.323000,light
737440,us6000ddq7,2021-02-01 16:55:28.961,"35 km E of Chipinge, Zimbabwe",-20.1780,32.9603,10.000,4.3,284,nan,ZWE,2021-02-01,16:55:28.961000,light
737441,us6000f3h6,2021-08-09 01:39:40.685,"9 km ESE of Binga, Zimbabwe",-17.6416,27.4238,10.000,4.3,284,nan,ZWE,2021-08-09,01:39:40.685000,light
737442,us7000hc68,2022-05-24 09:14:59.331,"14 km SE of Binga, Zimbabwe",-17.6952,27.4567,10.000,4.7,343,nan,ZWE,2022-05-24,09:14:59.331000,light


In [32]:
import os
 
list = [f'eq_events/eq_events_{year}' for year in range(1989, 2025)]
 
for items in list:
    os.mkdir(items)

In [22]:
g = df.groupby(pd.Grouper(key='datetime', freq='M'))
# groups to a list of dataframes with list comprehension
dfs = [group for _,group in g]

/tmp/ipykernel_2070/3224197441.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  g = df.groupby(pd.Grouper(key='datetime', freq='M'))


In [28]:
dfs[100]['date'].to_list()[0].strftime("%m")

'05'

In [33]:
for i in range(0,423):
    year = dfs[i]["date"].to_list()[0].strftime("%Y")
    month = dfs[i]["date"].to_list()[0].strftime("%m")
    dfs[i].to_csv(f'eq_events/eq_events_{year}/eq_events_{year}_{month}.csv')

In [24]:
str(dfs[0]["date"])[50]

' '

In [25]:
dfs[0]

,event_id,datetime,location,latitude,longitude,depth,magnitude,significance,alert,country,date,timestamp,level
9737,usp0003qd3,1989-01-01 05:55:45.860,"4 km WNW of Asprángeloi, Greece",39.842,20.684,10.0,3.3,168,nan,ALB,1989-01-01,05:55:45.860000,minor
250258,usp0003qd3,1989-01-01 05:55:45.860,"4 km WNW of Asprángeloi, Greece",39.842,20.684,10.0,3.3,168,nan,GRC,1989-01-01,05:55:45.860000,minor
720490,usp0003qd4,1989-01-01 07:03:27.520,"84 km NW of Speightstown, Barbados",13.884,-60.080,33.0,3.8,222,nan,VEN,1989-01-01,07:03:27.520000,minor
290042,usp0003qd5,1989-01-01 07:10:01.760,"north of Halmahera, Indonesia",3.013,128.039,140.0,4.0,246,nan,IDN,1989-01-01,07:10:01.760000,light
290043,usp0003qd7,1989-01-01 08:41:38.630,"223 km WNW of Tual, Indonesia",-4.758,130.934,33.0,4.3,284,nan,IDN,1989-01-01,08:41:38.630000,light
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11988,usp0003re7,1989-01-31 23:23:13.480,"53 km ENE of Machalí, Chile",-33.964,-70.127,127.1,4.7,340,nan,ARG,1989-01-31,23:23:13.480000,light
134865,usp0003re7,1989-01-31 23:23:13.480,"53 km ENE of Machalí, Chile",-33.964,-70.127,127.1,4.7,340,nan,CHL,1989-01-31,23:23:13.480000,light
469934,usp0003re7,1989-01-31 23:23:13.480,"53 km ENE of Machalí, Chile",-33.964,-70.127,127.1,4.7,340,nan,NZL,1989-01-31,23:23:13.480000,light
290133,usp0003re8,1989-01-31 23:51:34.430,"248 km ESE of Biak, Indonesia",-1.940,138.185,33.0,4.5,312,nan,IDN,1989-01-31,23:51:34.430000,light
